***Converting between clicks and states in a one-column puzzle***

In [9]:
#clicks: x + 4x^2 + 3x^3
#states: 1 + 3x^2 + 2x^3 + 3x^4
R.<x> = Zmod(5)[]
(x + 4*x^2 + 3*x^3 + 3*x^4)*(x^(-1) + 1 + x)

3*x^5 + x^4 + 3*x^2 + 1

In [13]:
#clicks: x + 3x^2 + 2x^4 + 4x^5
#states: 1 + 4x + 4x^2 + x^4 + x^5
R.<x> = Zmod(5)[]
-R((x + 3*x^2 + 2*x^4 + 4*x^5)*(x^(-1)+1+x)) % (x^6 - 1)

4*x^5 + 4*x^4 + x^2 + x

# Functions for generating the polynomials $p_j$

In [1]:
dict_p = {} #We use memoization to find the polynomials we need

#This function generates the polynomials, with integer coefficients (not mod k)
def p_n_nokM(n):
    R.<x> = ZZ[]
    first = [0, 1, -x, x^2 - 1]

    #Base cases
    if n <= 3:
        return first[n]
    
    #Start the array of indices needed
    indicesNeeded = [n]
    placeInList = -1
    curIndex = n
    
    #Go through the array and get all the indices needed
    while curIndex > 3:
        placeInList += 1
        curIndex = indicesNeeded[placeInList]
        
        # if the two indices below the current one are also needed, skip this one (we'll get it by recursive def.)
        if curIndex - 1 in indicesNeeded and curIndex - 2 in indicesNeeded:
            continue
        
        # Check parity and add the indices into the list, in decreasing order
        half = curIndex // 2
        if curIndex % 2:
            if half+1 not in indicesNeeded:
                indicesNeeded.append(half+1)
            if half not in indicesNeeded:
                indicesNeeded.append(half)
        else:
            if half+1 not in indicesNeeded:
                indicesNeeded.append(half+1)
            if half not in indicesNeeded:
                indicesNeeded.append(half)
            if half-1 not in indicesNeeded:
                indicesNeeded.append(half-1)
        
    #Initialize a dictionary that will store the results
    dict_p_n_nokM = {}
    for i in range(4):
        dict_p_n_nokM[i] = first[i]
    
    #Go back through the list backwards and compute each index
    for i in range(1, len(indicesNeeded)+1):
        curIndex = indicesNeeded[-i]
        if curIndex - 1 in dict_p_n_nokM.keys() and curIndex - 2 in dict_p_n_nokM.keys():
            dict_p_n_nokM[curIndex] = -x*dict_p_n_nokM[curIndex-1] - dict_p_n_nokM[curIndex-2]
            continue
        
        half = curIndex // 2
        if curIndex % 2:
            dict_p_n_nokM[curIndex] = (dict_p_n_nokM[half+1])^2 - (dict_p_n_nokM[half])^2
        else:
            dict_p_n_nokM[curIndex] = dict_p_n_nokM[half] * dict_p_n_nokM[half+1] - dict_p_n_nokM[half-1] * dict_p_n_nokM[half]
            
    return dict_p_n_nokM[n]

#This function generates the polynomials (mod k)
def p_n_noM(n, k):
    if k:
        R.<x> = Zmod(k)[]
    else:
        R.<x> = ZZ[]
    
    #Base cases
    if n <= 5:
        return (linear_p_n(n, k))
    
    #Start the array of indices needed
    indicesNeeded = [n]
    placeInList = -1
    curIndex = n
    
    #Go through the array and get all the indices needed
    while curIndex > 9:
        placeInList += 1
        curIndex = indicesNeeded[placeInList]
        
        # if the two indices below the current one are also needed, skip this one (we'll get it by recursive def.)
        if curIndex - 1 in indicesNeeded and curIndex - 2 in indicesNeeded:
            continue
        
        # Check parity and add the indices into the list, in decreasing order
        half = curIndex // 2
        if curIndex % 2:
            if half+1 not in indicesNeeded:
                indicesNeeded.append(half+1)
            if half not in indicesNeeded:
                indicesNeeded.append(half)
        else:
            if half+1 not in indicesNeeded:
                indicesNeeded.append(half+1)
            if half not in indicesNeeded:
                indicesNeeded.append(half)
            if half-1 not in indicesNeeded:
                indicesNeeded.append(half-1)
        
    #Initialize a dictionary that will store the results
    dict_p_n_noM = {}
    for i in range(6):
        dict_p_n_noM[i] = linear_p_n(i, k)
    
    #Go back through the list backwards and compute each index
    for i in range(1, len(indicesNeeded)+1):
        curIndex = indicesNeeded[-i]
        if curIndex - 1 in dict_p_n_noM.keys() and curIndex - 2 in dict_p_n_noM.keys():
            dict_p_n_noM[curIndex] = -x*dict_p_n_noM[curIndex-1] - dict_p_n_noM[curIndex-2]
            continue
        
        half = curIndex // 2
        if curIndex % 2:
            dict_p_n_noM[curIndex] = (dict_p_n_noM[half+1])^2 - (dict_p_n_noM[half])^2
        else:
            dict_p_n_noM[curIndex] = dict_p_n_noM[half] * dict_p_n_noM[half+1] - dict_p_n_noM[half-1] * dict_p_n_noM[half]
    
    return dict_p_n_noM[n]

#This function is used when computing p_j(A)e_1; this takes p_j % χ_j
#where χ_j is the characteristic polynomial of the matrix A = A_M
#We probably won't use this function much
def p_n(n, k, M):
    R.<x> = Zmod(k)[]
    chi = chi_n(M+1, k)
    
    #Base cases
    if n <= 5:
        return (linear_p_n(n, k) % chi)
    
    #Start the array of indices needed
    indicesNeeded = [n]
    placeInList = -1
    curIndex = n
    
    #Go through the array and get all the indices needed
    while curIndex > 9:
        placeInList += 1
        curIndex = indicesNeeded[placeInList]
        
        # if the two indices below the current one are also needed, skip this one (we'll get it by recursive def.)
        if curIndex - 1 in indicesNeeded and curIndex - 2 in indicesNeeded:
            continue
        
        # Check parity and add the indices into the list, in decreasing order
        half = curIndex // 2
        if curIndex % 2:
            if half+1 not in indicesNeeded:
                indicesNeeded.append(half+1)
            if half not in indicesNeeded:
                indicesNeeded.append(half)
        else:
            if half+1 not in indicesNeeded:
                indicesNeeded.append(half+1)
            if half not in indicesNeeded:
                indicesNeeded.append(half)
            if half-1 not in indicesNeeded:
                indicesNeeded.append(half-1)
        
    #Initialize a dictionary that will store the results
    dict_p_n = {}
    for i in range(6):
        dict_p_n[i] = linear_p_n(i, k) % chi
    
    #Go back through the list backwards and compute each index
    for i in range(1, len(indicesNeeded)+1):
        curIndex = indicesNeeded[-i]
        if curIndex - 1 in dict_p_n.keys() and curIndex - 2 in dict_p_n.keys():
            dict_p_n[curIndex] = (-x*dict_p_n[curIndex-1] - dict_p_n[curIndex-2]) % chi
            continue
        
        half = curIndex // 2
        if curIndex % 2:
            dict_p_n[curIndex] = ((dict_p_n[half+1])^2 - (dict_p_n[half])^2) % chi
        else:
            dict_p_n[curIndex] = (dict_p_n[half] * dict_p_n[half+1] - dict_p_n[half-1] * dict_p_n[half]) % chi
    
    return dict_p_n[n]
    
#This is a bad implementation, using recursion instead of storing things in a dictionary
def old_p_n(n, k, M):    
    R.<x> = Zmod(k)[]
    if (n, k, M) in dict_p.keys():
        return dict_p[(n, k, M)]
    
    chi_M = chi_n(M+1, k)
    
    #Base cases of recursion
    if n <= 5:
        return (linear_p_n(n, k) % chi_M)
    
    if n % 2:
        ans = (p_n(n//2+1, k, M)^2 - p_n(n//2, k, M)^2) % chi_M
        dict_p[(n, k, M)] = ans
        return ans
    else:
        ans = (p_n(n//2, k, M) * p_n(n//2+1, k, M) - p_n(n//2 - 1, k, M) * p_n(n//2, k, M)) % chi_M
        dict_p[(n, k, M)] = ans
        return ans

#This is not an efficient function for large n
def linear_p_n(n, k):
    R.<x> = Zmod(k)[]
    prev = 0
    cur = 1
    for i in range(n):
        new = -x*cur - prev
        prev = cur
        cur = new
    return prev

#This computes the characteristic polynomial of A = A_n
#Another formula for this polynomial: χ_n(n, k) = p_n_noM(n, k)(x-1)
def chi_n(n, k=0):
    if not k:
        R.<x> = ZZ[]
    else:
        R.<x> = Zmod(k)[]
    prev = 0
    cur = 1
    for i in range(n):
        new = (x-1)*cur - prev
        prev = cur
        cur = new
    return prev

# from time import time
# start = time()
# print(p_n(10034578923408923705973495871295723049872896740957820598347089572096847256345234567, 5, 100))
# print()

print("Loaded basic commands.")

Loaded basic commands.


In [7]:
chi_n(4)

for m in range(1,11):
    for n in range (1,11):
        print(chi_n(m+n)==chi_n(m)*chi_n(n+1)-chi_n(m-1) * chi_n(n))

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [3]:
#Example computation showing that o(11, 5) divides 780
p = 5
R.<x> = Zmod(p)[]
f = p_n_noM(780, p)
factor(f(x^(-1)+1+x))
M = 4
poly = (x^(2*M+2)-1)/(x^2-1)
#f % poly?
R(f*x^779) % poly

0

In [4]:
#Example computation showing that o(4, 5) divides 15
p = 5
R.<x> = Zmod(p)[]
f = p_n_noM(15, p)
print(factor(f(x^(-1)+1+x)))
M = 4
print((x^(2*M+2)-1)/(x^2-1))
print("Note that all the factors in the bottom polynomial are also found in the top polynomial, so the top one is divisible by the bottom one.")

x^-14 * (x + 4)^4 * (x + 2)^5 * (x + 3)^5 * (x + 1)^10 * (x^2 + 4*x + 1)^2
x^8 + x^6 + x^4 + x^2 + 1
Note that all the factors in the bottom polynomial are also found in the top polynomial, so the top one is divisible by the bottom one.


In [5]:
p = 7
R.<x> = Zmod(p)[]
f = p_n_noM(20, p)
print(factor(f(x^(-1)+1+x)))

(6) * x^-19 * (x + 3) * (x + 5) * (x + 6)^2 * (x^2 + 4*x + 1) * (x^4 + x^3 + x^2 + x + 1) * (x^4 + 2*x^3 + 2*x + 5) * (x^4 + 3*x^3 + 3*x^2 + 3*x + 1) * (x^4 + 6*x^3 + 6*x + 3) * (x^8 + 2*x^6 + 5*x^5 + 5*x^3 + 2*x^2 + 1) * (x^8 + 4*x^7 + 5*x^6 + 6*x^5 + 2*x^4 + 6*x^3 + 5*x^2 + 4*x + 1)


In [5]:
p = 2
R.<x> = Zmod(p)[]
f = p_n_noM(72, p)
print(factor(f(x^(-1)+1+x)))
M = 55
poly = (x^(2*M+2)-1)/(x^2-1)
print(R(f)%poly)
print(factor(poly))

x^-71 * (x + 1)^32 * (x^2 + x + 1)^7 * (x^3 + x + 1)^16 * (x^3 + x^2 + 1)^16
0
(x + 1)^14 * (x^3 + x + 1)^16 * (x^3 + x^2 + 1)^16


In [4]:
p = 3
R.<x> = Zmod(p)[]
f = p_n_noM(122, p)
print(factor(f(x^(-1)+1+x)))
M = 10
poly = ((x^(2*M+2)-1)/(x^2-1))
print(R(f)%poly)
print(factor(poly))

(2) * x^-121 * (x + 2)^2 * (x^5 + x^2 + x + 2) * (x^5 + 2*x^2 + x + 1) * (x^5 + x^3 + x + 1) * (x^5 + x^3 + x + 2) * (x^5 + x^3 + x^2 + 2*x + 2) * (x^5 + x^3 + 2*x^2 + 2*x + 1) * (x^5 + 2*x^3 + x^2 + 2*x + 2) * (x^5 + 2*x^3 + 2*x^2 + 2*x + 1) * (x^5 + x^4 + x^2 + 1) * (x^5 + x^4 + x^2 + x + 1) * (x^5 + x^4 + x^3 + x + 1) * (x^5 + x^4 + x^3 + x^2 + 2*x + 1) * (x^5 + x^4 + x^3 + 2*x^2 + x + 2) * (x^5 + x^4 + 2*x^3 + 1) * (x^5 + x^4 + 2*x^3 + x^2 + 2) * (x^5 + x^4 + 2*x^3 + 2*x^2 + 2) * (x^5 + 2*x^4 + 2*x^2 + 2) * (x^5 + 2*x^4 + 2*x^2 + x + 2) * (x^5 + 2*x^4 + x^3 + x + 2) * (x^5 + 2*x^4 + x^3 + x^2 + x + 1) * (x^5 + 2*x^4 + x^3 + 2*x^2 + 2*x + 2) * (x^5 + 2*x^4 + 2*x^3 + 2) * (x^5 + 2*x^4 + 2*x^3 + x^2 + 1) * (x^5 + 2*x^4 + 2*x^3 + 2*x^2 + 1) * (x^10 + x^7 + x^5 + x^3 + 1) * (x^10 + 2*x^7 + x^6 + 2*x^5 + x^4 + 2*x^3 + 1) * (x^10 + x^8 + x^7 + 2*x^6 + x^5 + 2*x^4 + x^3 + x^2 + 1) * (x^10 + x^8 + 2*x^7 + 2*x^5 + 2*x^3 + x^2 + 1) * (x^10 + 2*x^8 + 2*x^7 + 2*x^6 + 2*x^5 + 2*x^4 + 2*x^3 + 2*x

In [8]:
p = 7
R.<x> = Zmod(p)[]
f = p_n_noM(84, p)
print(factor(f(x^(-1)+1+x)))
M = 13
print((x^(2*M+2)-1)/(x^2-1))

(6) * x^-83 * (x + 2)^3 * (x + 4)^3 * (x + 3)^7 * (x + 5)^7 * (x + 1)^14 * (x + 6)^14 * (x^2 + 3*x + 1)^3 * (x^2 + 1)^7 * (x^2 + x + 3)^7 * (x^2 + 4*x + 1)^7 * (x^2 + 4*x + 5)^7 * (x^2 + 5*x + 3)^7 * (x^2 + 5*x + 5)^7 * (x^4 + 5*x^3 + 5*x^2 + 5*x + 1)^7
x^26 + x^24 + x^22 + x^20 + x^18 + x^16 + x^14 + x^12 + x^10 + x^8 + x^6 + x^4 + x^2 + 1


In [6]:
p = 3
R.<x> = Zmod(p)[]
f = p_n_noM(18, p)
print(factor(f(x^(-1)+1+x)))
M = 5
poly = ((x^(2*M+2)-1)/(x^2-1))
print(R(f)%poly)
print(factor(poly))

(2) * x^-17 * (x + 1)^8 * (x + 2)^18 * (x^2 + 1)^4
0
(x + 1)^2 * (x + 2)^2 * (x^2 + 1)^3


In [7]:
p = 11
R.<x> = Zmod(p)[]
f = p_n_noM(165, p)
print(factor(f(x^(-1)+1+x)))
M = 10
poly = ((x^(2*M+2)-1)/(x^2-1))
print(R(f)%poly)
print(factor(poly))

x^-164 * (x + 5)^5 * (x + 9)^5 * (x + 2)^11 * (x + 6)^11 * (x + 7)^11 * (x + 8)^11 * (x + 1)^22 * (x + 10)^22 * (x^2 + 10*x + 1)^5 * (x^2 + 1)^11 * (x^2 + x + 7)^11 * (x^2 + 4*x + 8)^11 * (x^2 + 5*x + 1)^11 * (x^2 + 6*x + 7)^11 * (x^2 + 8*x + 8)^11 * (x^4 + 5*x^3 + x^2 + 5*x + 1)^11 * (x^4 + 6*x^3 + 9*x^2 + 6*x + 1)^11
0
(x + 1)^10 * (x + 10)^10


In [8]:
p = 13
R.<x> = Zmod(p)[]
f = p_n_noM(273, p)
print(factor(f(x^(-1)+1+x)))
M = 12
poly = ((x^(2*M+2)-1)/(x^2-1))
print(R(f)%poly)
print(factor(poly))

x^-272 * (x + 3)^6 * (x + 9)^6 * (x + 2)^13 * (x + 5)^13 * (x + 6)^13 * (x + 7)^13 * (x + 8)^13 * (x + 11)^13 * (x + 1)^26 * (x + 12)^26 * (x^2 + 3*x + 1)^6 * (x^2 + x + 2)^13 * (x^2 + 6*x + 1)^13 * (x^2 + 7*x + 1)^13 * (x^2 + 7*x + 7)^13 * (x^2 + 8*x + 1)^13 * (x^2 + 9*x + 2)^13 * (x^2 + 11*x + 7)^13 * (x^4 + 5*x^3 + 12*x^2 + 5*x + 1)^13 * (x^4 + 9*x^3 + 4*x^2 + 9*x + 1)^13 * (x^4 + 10*x^3 + 7*x^2 + 10*x + 1)^13 * (x^4 + 12*x^3 + 6*x^2 + 12*x + 1)^13
0
(x + 1)^12 * (x + 12)^12


In [10]:
def w(p,M):
    return p*(p^(2*euler_phi(M+1))-1)
p = 7
M = 4
#w(p,M)

R.<x> = Zmod(p)[]
f = p_n_noM(w(p,M), p)
print((f%(x-1)^((p-1)/2)))
#print((f%(x^2-2*x-1)^(p)))
#print(factor(f))
poly = chi_n(M)
#((x^(2*M+2)-1)/(x^2-1))
#print(R(f)%poly)
#print(factor(poly))

2*x^6 + 2*x^3 + 1


In [ ]:
(x^2 + 1) * (x^2 + x + 2) * (x^2 + 2*x + 2) * (x^4 + x^2 + 2) * (x^4 + 2*x^2 + 2)

In [7]:
y = x^-1+1+x
g = x-1
factor(g(y))

x^-1 * (x + 2) * (x + 3)

In [13]:
P = 5 
R.<x> = ZZ[]

for M in range(2,41):
#     if M %5 == 4:
#         continue
    print(M,factor((x^(2*M+2)-1)/(x^2-1)))
# Proposition: (x+2)*(x+3) is a factor of I(x) precisely when M is odd
# proof: (x+2)*(x+3) = (x^2+5x+6)%5 = x^2 + 1
# When M is odd, M+1 = 2s for some s.  
# (x^(2*M+2)-1) = (x^(2*(2s))-1) = (x^4s-1) = (x^4-1)(stuff) = (x^2-1)(x^2+1)(stuff)

#(x^2 + x + 1) * (x^2 + 4*x + 1) is a factor for M = 2,8,14,20,26. Hint: x^3-1 = (x-1)(x^2+x+1) and x^3+1 = (x+1)*(x^2-x+1)


(2, (x^2 - x + 1) * (x^2 + x + 1))
(3, (x^2 + 1) * (x^4 + 1))
(4, (x^4 - x^3 + x^2 - x + 1) * (x^4 + x^3 + x^2 + x + 1))
(5, (x^2 - x + 1) * (x^2 + 1) * (x^2 + x + 1) * (x^4 - x^2 + 1))
(6, (x^6 - x^5 + x^4 - x^3 + x^2 - x + 1) * (x^6 + x^5 + x^4 + x^3 + x^2 + x + 1))
(7, (x^2 + 1) * (x^4 + 1) * (x^8 + 1))
(8, (x^2 - x + 1) * (x^2 + x + 1) * (x^6 - x^3 + 1) * (x^6 + x^3 + 1))
(9, (x^2 + 1) * (x^4 - x^3 + x^2 - x + 1) * (x^4 + x^3 + x^2 + x + 1) * (x^8 - x^6 + x^4 - x^2 + 1))
(10, (x^10 - x^9 + x^8 - x^7 + x^6 - x^5 + x^4 - x^3 + x^2 - x + 1) * (x^10 + x^9 + x^8 + x^7 + x^6 + x^5 + x^4 + x^3 + x^2 + x + 1))
(11, (x^2 - x + 1) * (x^2 + 1) * (x^2 + x + 1) * (x^4 - x^2 + 1) * (x^4 + 1) * (x^8 - x^4 + 1))
(12, (x^12 - x^11 + x^10 - x^9 + x^8 - x^7 + x^6 - x^5 + x^4 - x^3 + x^2 - x + 1) * (x^12 + x^11 + x^10 + x^9 + x^8 + x^7 + x^6 + x^5 + x^4 + x^3 + x^2 + x + 1))
(13, (x^2 + 1) * (x^6 - x^5 + x^4 - x^3 + x^2 - x + 1) * (x^6 + x^5 + x^4 + x^3 + x^2 + x + 1) * (x^12 - x^10 + x^8 - x^6 + x^4 

In [1]:
euler_phi(7432339208719)

7432339208718